In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklear.model_selection import train_test_split

In [79]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
# Max entry is 255
X_train_full, X_test = X_train_full/255, X_test/255

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2)

In [83]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((40000, 32, 32, 3), (10000, 32, 32, 3), (40000, 1), (10000, 1))

In [102]:
def build_model(BN: bool, lr, neurons: int, layerz: int, activation: str, initializer: str):
    model = keras.Sequential([keras.layers.Flatten(input_shape=(32, 32, 3))])
    for _ in range(layerz):
        if BN: model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(neurons, activation = activation, kernel_initializer = initializer))
    if BN: model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(10, activation = 'softmax'))
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer = keras.optimizers.Nadam(lr=lr))
    return model

In [103]:
model_no_BN = build_model(BN=False, lr=0.001, 
                     neurons=100, layerz=20, activation='elu', initializer='he_normal')
# Default lr on Nadam is 0.001

In [98]:
model_no_BN.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100,
                callbacks = [keras.callbacks.EarlyStopping(patience=10), keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=8)])

Epoch 1/100
1250/1250 [==============================] - 12s 8ms/step - loss: 2.4103 - accuracy: 0.1815 - val_loss: 1.9752 - val_accuracy: 0.2990
Epoch 2/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.8913 - accuracy: 0.3117 - val_loss: 1.9911 - val_accuracy: 0.3102
Epoch 3/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.8114 - accuracy: 0.3473 - val_loss: 1.7998 - val_accuracy: 0.3634
Epoch 4/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.7600 - accuracy: 0.3685 - val_loss: 1.7249 - val_accuracy: 0.3897
Epoch 5/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.7085 - accuracy: 0.3880 - val_loss: 1.6542 - val_accuracy: 0.4104
Epoch 6/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.6712 - accuracy: 0.4002 - val_loss: 1.6679 - val_accuracy: 0.4097
Epoch 7/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.6368 - accuracy: 0.4193 - val_loss: 1.7388 - val_a

In [100]:
model_no_BN.evaluate(X_test, y_test)[1]

313/313 [==============================] - 0s 1ms/step - loss: 1.6107 - accuracy: 0.4202


0.420199990272522

In [119]:
evals = []
for k in range(2, 5):
    model = build_model(BN=False, lr=k/10**4, 
                     neurons=100, layerz=20, activation='elu', initializer='he_normal')
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100,
                callbacks = [keras.callbacks.EarlyStopping(patience=10)])
    evals.append((model.evaluate(X_test, y_test)[1], k/10**4))
    print(evals)

Epoch 1/100
1250/1250 [==============================] - 13s 8ms/step - loss: 2.1250 - accuracy: 0.2267 - val_loss: 1.8436 - val_accuracy: 0.3470
Epoch 2/100
1250/1250 [==============================] - 9s 8ms/step - loss: 1.7585 - accuracy: 0.3608 - val_loss: 1.6857 - val_accuracy: 0.3962
Epoch 3/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.6496 - accuracy: 0.4054 - val_loss: 1.5984 - val_accuracy: 0.4248
Epoch 4/100
1250/1250 [==============================] - 9s 8ms/step - loss: 1.5857 - accuracy: 0.4316 - val_loss: 1.5913 - val_accuracy: 0.4186
Epoch 5/100
1250/1250 [==============================] - 9s 8ms/step - loss: 1.5293 - accuracy: 0.4491 - val_loss: 1.5582 - val_accuracy: 0.4456
Epoch 6/100
1250/1250 [==============================] - 10s 8ms/step - loss: 1.4883 - accuracy: 0.4650 - val_loss: 1.5018 - val_accuracy: 0.4684
Epoch 7/100
1250/1250 [==============================] - 11s 9ms/step - loss: 1.4427 - accuracy: 0.4821 - val_loss: 1.5208 - val

KeyboardInterrupt: 

In [121]:
evals_no_BN = evals1 # ==> best lr is 2/10**4
evals_no_BN

[(0.10000000149011612, 0.1),
 (0.10000000149011612, 0.01),
 (0.3806999921798706, 0.001),
 (0.4918999969959259, 0.0001),
 (0.490200012922287, 0.0001),
 (0.48590001463890076, 1e-05),
 (0.4327000081539154, 1e-06),
 (0.4941999912261963, 0.0002)]

## Now using batch norm:

In [125]:
evals = []
for k in range(3, 6, 2):
    model = build_model(BN=True, lr=k/10**3, 
                     neurons=100, layerz=20, activation='elu', initializer='he_normal')
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100,
                callbacks = [keras.callbacks.EarlyStopping(patience=10)])
    evals.append((model.evaluate(X_test, y_test)[1], k/10**3))
    print(evals)

Epoch 1/100
1250/1250 [==============================] - 21s 11ms/step - loss: 2.0670 - accuracy: 0.2576 - val_loss: 1.7961 - val_accuracy: 0.3603
Epoch 2/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.7834 - accuracy: 0.3657 - val_loss: 1.7436 - val_accuracy: 0.3779
Epoch 3/100
1250/1250 [==============================] - 13s 11ms/step - loss: 1.7106 - accuracy: 0.3921 - val_loss: 1.6809 - val_accuracy: 0.3923
Epoch 4/100
1250/1250 [==============================] - 13s 10ms/step - loss: 1.6451 - accuracy: 0.4120 - val_loss: 1.6209 - val_accuracy: 0.4291
Epoch 5/100
1250/1250 [==============================] - 13s 10ms/step - loss: 1.6134 - accuracy: 0.4269 - val_loss: 1.6207 - val_accuracy: 0.4256
Epoch 6/100
1250/1250 [==============================] - 13s 10ms/step - loss: 1.5650 - accuracy: 0.4461 - val_loss: 1.5441 - val_accuracy: 0.4598
Epoch 7/100
1250/1250 [==============================] - 13s 10ms/step - loss: 1.5279 - accuracy: 0.4622 - val_loss: 1

1250/1250 [==============================] - 14s 11ms/step - loss: 1.1274 - accuracy: 0.6054 - val_loss: 1.4410 - val_accuracy: 0.5053
Epoch 29/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.1109 - accuracy: 0.6083 - val_loss: 1.6257 - val_accuracy: 0.4676
Epoch 30/100
1250/1250 [==============================] - 13s 11ms/step - loss: 1.1037 - accuracy: 0.6097 - val_loss: 1.7571 - val_accuracy: 0.4002
Epoch 31/100
1250/1250 [==============================] - 13s 11ms/step - loss: 1.1035 - accuracy: 0.6128 - val_loss: 1.4711 - val_accuracy: 0.5113
Epoch 32/100
1250/1250 [==============================] - 13s 11ms/step - loss: 1.0841 - accuracy: 0.6214 - val_loss: 1.4674 - val_accuracy: 0.5087
Epoch 33/100
1250/1250 [==============================] - 13s 11ms/step - loss: 1.0774 - accuracy: 0.6219 - val_loss: 1.5502 - val_accuracy: 0.4866
Epoch 34/100
1250/1250 [==============================] - 13s 11ms/step - loss: 1.0768 - accuracy: 0.6217 - val_loss: 1.5188 

In [128]:
evals_BN ==>  # 0.001 is best lr

[(0.5238000154495239, 0.001),
 (0.5127999782562256, 0.0001),
 (0.5097000002861023, 0.003),
 (0.49810001254081726, 0.005),
 (0.5097000002861023, 0.003),
 (0.49810001254081726, 0.005)]